# Chatbot pentru răspunderea la întrebări frecvente



## Instalarea bibliotecilor necesare

In [13]:
# daca nu sunt deja instalate
# altfel se comenteaza
%pip install numpy
%pip install scikit-learn
%pip install scipy
%pip install nltk
%pip install pandas
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\akopo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\akopo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\akopo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\akopo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\akopo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Prelucrarea datelor

1. Impartirea setului de date pentru antrenarea modelului

In [14]:
import pandas as pd

df = pd.read_csv("all_questions.txt", sep="\t")
df.head()

print(df.columns)

data = df[['Question', 'Answer']].dropna()

print(data)

from sklearn.model_selection import train_test_split

train, tmp = train_test_split(data, test_size=0.3, random_state=42)

val, test = train_test_split(tmp, test_size=0.5, random_state=42)

print(f"Train:{len(train)}")
print(f"Valid:{len(val)}")
print(f"Test:{len(test)}")

Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')
                                               Question  \
0     Was Abraham Lincoln the sixteenth President of...   
1     Was Abraham Lincoln the sixteenth President of...   
2     Did Lincoln sign the National Banking Act of 1...   
3     Did Lincoln sign the National Banking Act of 1...   
4                      Did his mother die of pneumonia?   
...                                                 ...   
3992          What areas do the Grevy's Zebras inhabit?   
3994  Which species of zebra is known as the common ...   
3995  Which species of zebra is known as the common ...   
3996                     At what age can a zebra breed?   
3997                     At what age can a zebra breed?   

                                                 Answer  
0                                                   yes  
1                              

2. Prelucrarea textului

Constă în:
* eliminarea duplicatelor,
* transformarea din uppercase în lowercase,
* eliminarea caracterelor care nu sunt cuvinte și care nu sunt whitespace-uri
* eliminarea cifrelor din text.

In [26]:
# eliminare duplicate

train["Question"] = train["Question"].drop_duplicates(keep="first")
val["Question"] = val["Question"].drop_duplicates(keep="first")
test["Question"] = test["Question"].drop_duplicates(keep="first")

print(f"Train:{len(train)}")
print(f"Valid:{len(val)}")
print(f"Test:{len(test)}")

# transformare din uppercase in lowercase

train["Question"] = train["Question"].map(lambda x: x.lower() if isinstance(x, str) else x)
val["Question"] = val["Question"].map(lambda x: x.lower() if isinstance(x, str) else x)
test["Question"] = test["Question"].map(lambda x: x.lower() if isinstance(x, str) else x)

# eliminare a caracterelor care nu sunt cuvinte si care nu sunt whitespace-uri

train["Question"] = train["Question"].replace(to_replace=r'[^\w\s]', value='', regex=True)
val["Question"] = val["Question"].replace(to_replace=r'[^\w\s]', value='', regex=True)
test["Question"] = test["Question"].replace(to_replace=r'[^\w\s]', value='', regex=True)

# eliminare cifre din text 

train["Question"] = train["Question"].replace(to_replace=r'\d', value='', regex=True)
val["Question"] = val["Question"].replace(to_replace=r'\d', value='', regex=True)
test["Question"] = test["Question"].replace(to_replace=r'\d', value='', regex=True)

train = train.dropna(subset=["Question"])
val = val.dropna(subset=["Question"])
test = test.dropna(subset=["Question"])

print(f"Text curatat:")
print(train["Question"])

Train:2240
Valid:505
Test:508
Text curatat:
2954              [drums, are, usually, played, by, what]
3359    [were, the, recitations, of, the, ancient, gre...
1214                      [are, otters, playful, animals]
601     [is, ford, one, of, only, four, former, presid...
3855    [who, were, the, orkhon, inscriptions, built, ...
                              ...                        
603     [did, grand, rapids, supporters, not, urge, hi...
3633    [what, kind, of, piano, did, irving, berlin, p...
1315                                  [garbage, is, what]
1350    [will, oil, and, gas, probably, remain, the, b...
3715    [how, large, is, the, population, of, san, fra...
Name: Question, Length: 1798, dtype: object


3. Curatarea textului și vectorizarea TF-IDF

In [27]:
# tokenizare

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer(r'\w+')

train["Question"] = train["Question"].apply(lambda x: tokenizer.tokenize(x) if isinstance(x, str) else x)
val["Question"] = val["Question"].apply(lambda x: tokenizer.tokenize(x) if isinstance(x, str) else x)
test["Question"] = test["Question"].apply(lambda x: tokenizer.tokenize(x) if isinstance(x, str) else x)

# stergere stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

train["Question"] = train["Question"].apply(lambda x: [word for word in x if word not in stop_words])
val["Question"] = val["Question"].apply(lambda x: [word for word in x if word not in stop_words])
test["Question"] = test["Question"].apply(lambda x: [word for word in x if word not in stop_words])

                                               Question  \
2954                           [drums, usually, played]   
3359  [recitations, ancient, greeks, accompanied, ly...   
1214                         [otters, playful, animals]   
601   [ford, one, four, former, presidents, live, ye...   
3855                      [orkhon, inscriptions, built]   
...                                                 ...   
603   [grand, rapids, supporters, urge, take, bartel...   
3633                [kind, piano, irving, berlin, play]   
1315                                          [garbage]   
1350  [oil, gas, probably, remain, backbone, qatar, ...   
3715                [large, population, san, francisco]   

                                                 Answer  
2954                 the hands, or by one or two sticks  
3359                                                Yes  
1214                                                yes  
601                                                Yes.  
3

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akopo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

# lemmatizare
# initializare lemmatizer
lemmatizer = WordNetLemmatizer()

# functie pentru lemmatizare token-uri
def lemmatize_tokens(tokens):
    # conversie din pos in wordnet
    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    
    # lematizare token-uri
    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    
    # returnare ca o lista
    return lemmas

train["Question"] = train["Question"].apply(lemmatize_tokens)
val["Question"] = val["Question"].apply(lemmatize_tokens)
test["Question"] = test["Question"].apply(lemmatize_tokens)

print(train)